In [1]:
! pip install fredpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import seaborn as sns
import fredpy as fp
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing
from statsmodels.formula.api import ols
import yfinance as yf
import datetime as dt

In [3]:
fp.api_key='11be92f74470a93988d7ee70fa9b9cae'

In [4]:
#functions used in this notebook



#expanding window z-score so that there is no look-ahead bias in the z-score
def expanding_z_score(series, warmup=12):
    """
    Series of z-scores in an expanding window with a warmup period default equal to 12 periods.
    series(pd.Series): series to input
    warmup(int): period for warmum with na values
    Returns:
        pd.Series
    """
    average=series.expanding(min_periods=warmup).average()
    std=series.expanding(min_periods=warmup).std()
    z=(series-median)/std
    return z



def trim_z_scores(series,lower=-3,upper=+3):
    """Trims z-score of series to specified values"""

    series.loc[series < lower]=-3
    series.loc[series > upper]=3
            
    return series


def annualized_return(r,per):
    """ Annualized return calculated from a pd.Series or DataFrame.r may contain null values at start series.
    Args:
        r(pd.Series or Dataframe):return per period in R-format (eg:0.005=0.5%)
        per(int): the period adjustment (12 for monthly periods, 52 for weekly periods, 250 for daily periods)
    Returns:
        Float: annualized return
    """
    yearfraq=(r.shape[0]-r.isna().sum())/per
    wealth=(1+r).prod()
    annual_return=wealth**(1/yearfraq)-1
    return annual_return  

In [5]:
#import economic time series and transform if necessary, used to construct US Macro Indicator

#recessions
recessions = fp.series('USREC').as_frequency(freq='M').data


#fedfund rate
fedfunds=fp.series('FEDFUNDS').as_frequency(freq='M').data




#cpi
cpi_data = fp.series('PPIACO').as_frequency(freq='M').data
cpi=cpi_data/cpi_data.shift(12)-1
cpi=cpi.resample('M').last()
cpi_trend=(cpi-cpi.shift(12)).rolling(3).mean()






#composites

lei=pd.read_csv('us_ism_lei_cpi.csv',index_col=0,parse_dates=True).lei.resample('MS').last()
lei_6m=lei/lei.shift(6)-1


oecd=fp.series('USALOLITOAASTSAM').as_frequency(freq='M').data
nfci=fp.series('NFCI').as_frequency(freq='M').data

#Monetary
baa_fed=fp.series('BAAFFM').as_frequency(freq='M').data
treasury=fp.series('DGS10').as_frequency(freq='M').data
tbill=fp.series('TB3MS').as_frequency(freq='M').data
fedfunds=fp.series('FEDFUNDS').as_frequency(freq='M').data

term_spread=treasury-tbill

baa=baa_fed+fedfunds
baa_spread=baa-treasury

#production
ism=pd.read_csv('us_ism_lei_cpi.csv',index_col=0,parse_dates=True).ism_manu.resample('MS').last()
manufacturing_hours=fp.series('AWHMAN').as_frequency(freq='M').data

#consumer
consumer=fp.series('UMCSENT').as_frequency('M').data
consumer.fillna(method='ffill',inplace=True)

#employment
parttime=fp.series('LNS12032198').as_frequency('M').data
claims=fp.series('IC4WSA').as_frequency('M').data

#real estate
perm=fp.series('PERMIT').as_frequency(freq='M').data
starts=fp.series('HOUST').as_frequency(freq='M').data

In [6]:
# transform to series that will give z-score based on cycle if necessary, calculate z-scores

oecd_z=expanding_z_score(oecd,24).dropna()
nfci_z=expanding_z_score(nfci,24).dropna()*-1
lei_6m_z=expanding_z_score(lei_6m,24).dropna()


term_spread_z=expanding_z_score(term_spread,24).dropna()
baa_spread_z=expanding_z_score(baa_spread,24).dropna()*-1
ism_z=expanding_z_score(ism,24).dropna()
manufacturing_hours_z=expanding_z_score(manufacturing_hours,60).dropna()
consumer_z=expanding_z_score(consumer,24).dropna()
parttime_z=expanding_z_score(parttime,24).dropna()*-1
claims_z=expanding_z_score(claims,24).dropna()*-1
perm_z=expanding_z_score(perm,24).dropna()
starts_z=expanding_z_score(starts,24).dropna()

AttributeError: 'Expanding' object has no attribute 'average'

In [7]:
#trim z-scores

lei_6m_z_trim=trim_z_scores(lei_6m_z)
oecd_z_trim=trim_z_scores(oecd_z)
nfci_z_trim=trim_z_scores(nfci_z)

term_spread_z_trim=trim_z_scores(term_spread_z)
baa_spread_z_trim=trim_z_scores(baa_spread_z)

ism_z_trim=trim_z_scores(ism_z)
manufacturing_hours_z_trim=trim_z_scores(manufacturing_hours_z)

consumer_z_trim=trim_z_scores(consumer_z)

parttime_z_trim=trim_z_scores(parttime_z)
claims_z_trim=trim_z_scores(claims_z)

perm_z_trim=trim_z_scores(perm_z)
starts_z_trim=trim_z_scores(starts_z)

In [8]:
#Df without composites

z_df=pd.DataFrame({'lei_6m_z':lei_6m_z_trim,
                    'oecd_z':oecd_z_trim,
                   'nfci_z':nfci_z_trim,
                   'term_spread_z':term_spread_z_trim,
                   'baa_spread_z':baa_spread_z_trim,
                   'ism_z':ism_z,
                   'manufacturing_hours_z':manufacturing_hours_z_trim,
                   'consumer_z':consumer_z_trim,
                   'parttime_z':parttime_z_trim,
                   'claims_z':claims_z_trim,
                   'perm_z':perm_z_trim,
                   'starts_z':starts_z_trim                  
                  })

#sresample to end of month date and forward fill so that last known values are previous values, and start in 1955
z_df=z_df.resample('M').last()
z_df=z_df.fillna(method='ffill')
z_df=z_df['1955':]

#the raw indicator
macro_indicator_raw=z_df.median(axis=1)
#smooth with HP filter
cycle, macro_indicator_smooth = sm.tsa.filters.hpfilter(macro_indicator_raw,100)

In [9]:
z_df

In [10]:
#import spx and calc returns
spx=yf.download('^GSPC',start='1940-01-31')['Adj Close'].pct_change()+1
spx_monthly=spx.resample('M').apply(np.prod)
spx_1m=spx_monthly-1
spx_1m.index=spx_1m.index.tz_localize(None)
spx_1m=spx_1m.resample('M').last()


In [11]:
fig,ax=plt.subplots(figsize=(12,5))
sns.lineplot(macro_indicator_raw,color='red',linestyle='--',label='US Macro Cycle: raw',ax=ax)
sns.lineplot(macro_indicator_smooth,color='blue',linewidth=1,label='US Macro Cycle: smoothed (HP)',ax=ax)
ax.axhline(y=0,linestyle=':',color='red')
ax.set_title('US Macro-Cycle')
    
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=36))
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_xlabel('Date')
ax.set_ylabel('Macro-indicator')
sns.despine()
plt.legend()
plt.show()
fig.savefig('us_macro_cycle.png')

In [12]:
df=pd.DataFrame({
    'macro_raw':macro_indicator_raw,
    'macro_smooth':macro_indicator_smooth,
    'spx_1m':spx_1m
                      })
df=df['1955':]

In [13]:
df

In [14]:
df['macro_smooth_trend']=df.macro_smooth-df.macro_smooth.shift(3)
df=df.dropna()

In [15]:
df

In [16]:
conditions=[
    (df.macro_smooth > 0) & (df.macro_smooth_trend > 0),
    (df.macro_smooth > 0) & (df.macro_smooth_trend < 0),
    (df.macro_smooth < 0) & (df.macro_smooth_trend < 0),
    (df.macro_smooth < 0) & (df.macro_smooth_trend > 0)
    ]

choices=['expansion','slowdown','contraction','recovery']
df['regime']=np.select(conditions,choices)

In [17]:
fig,ax=plt.subplots(figsize=(5,8))

sns.boxplot(x='regime',y='spx_1m',data=df)
ax.set_title('Boxplot: monthly returns S&P500 by growth regimes',fontsize=10)

In [18]:
spx_regime=df.groupby('regime').spx_1m.mean()
spx_regime

In [19]:
df['spx_12m']=(df.spx_1m+1).rolling(12).apply(np.prod)-1
df.dropna(inplace=True)
df['spx_12m_z']=expanding_z_score(df.spx_12m,24)

In [20]:
model=ols('spx_1m ~ regime + 0',data=df).fit()
model.summary()

In [21]:
fig,ax=plt.subplots(figsize=(12,6))

sns.lineplot(df.macro_raw,color='red',linestyle=':',label='US Macro Cycle: raw',ax=ax)
sns.lineplot(df.macro_smooth,color='red',linewidth=1,label='US Macro Cycle: smoothed (HP)',ax=ax)
ax.axhline(y=0,linestyle=':',color='red')


ax2=ax.twinx()
sns.lineplot(df.spx_12m,alpha=0.5,label='S&P500 YoY%',ax=ax2)
ax2.axhline(y=0,linestyle='--')




ax.set_title('US Macro-Cycle')
    
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=36))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_xlabel('Date')
ax.set_ylabel('Macro-indicator')
sns.despine()
plt.legend()
plt.show()
fig.savefig('us_macro_cycle_spx.png')

In [22]:
fig,ax=plt.subplots(figsize=(12,6))

sns.lineplot(df.macro_raw,color='red',linestyle=':',label='US Macro Cycle: raw',ax=ax)
sns.lineplot(df.macro_smooth,color='red',linewidth=1,label='US Macro Cycle: smoothed (HP)',ax=ax)
ax.axhline(y=0,linestyle=':',color='red')



sns.lineplot(df.spx_12m_z,alpha=0.5,label='S&P500 YoY% z-score',ax=ax)





ax.set_title('US Macro-Cycle')
    
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=36))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_xlabel('Date')
ax.set_ylabel('Macro-indicator')
sns.despine()
plt.legend()
plt.show()
fig.savefig('us_macro_cycle_spx_z')

In [23]:
#download ff industries
datasets=web.famafrench.get_available_datasets()
dataset_name=[set for set in datasets if '17' in set and 'Industry' in set][0]

industry_dict=web.DataReader(dataset_name,'famafrench',start='1930-01-01')
industries=industry_dict[0]
industries.index=industries.index.astype('datetime64[ns]')
industries=industries.resample('M').last()
industries=industries/100

In [24]:
df_merged=pd.merge_ordered(df,industries,left_on=df.index,right_on=industries.index,how='left')
df_merged.dropna(inplace=True)
df_merged.set_index('key_0',inplace=True)
df_merged.index.name='date'
df_merged.columns=df_merged.columns.str.strip()
df_merged.columns=df_merged.columns.str.lower()
df_industries=df_merged.copy()

In [25]:
df_industries.head()

In [26]:
industry_list=df_industries.columns[7:]
industry_list=industry_list.insert(0,'spx_1m')

ind_grouped=df_industries.groupby('regime')[industry_list].mean()
ind_grouped=ind_grouped.transpose()
ind_grouped=np.round(ind_grouped*100,2)

In [27]:
ind_grouped.style.background_gradient(cmap="YlGn")

In [28]:

order_expansion=ind_grouped.sort_values(by='expansion',ascending=False).index
order_slowdown=ind_grouped.sort_values(by='slowdown',ascending=False).index
order_contraction=ind_grouped.sort_values(by='contraction',ascending=False).index
order_recovery=ind_grouped.sort_values(by='recovery',ascending=False).index


fig,ax=plt.subplots(4,1,figsize=(15,15))

sns.barplot(x=ind_grouped.index,y=ind_grouped.expansion,order=order_expansion,palette='summer',ax=ax[0])
ax[0].set_title('Average monthly performance  US industries during EXPANSION regimes (since 1960)')
ax[0].set_ylabel('monthly performance')

sns.barplot(x=ind_grouped.index,y=ind_grouped.slowdown,order=order_slowdown,palette='summer',ax=ax[1])
ax[1].set_title('Average monthly performance  US industries during SLOWDOWN regimes (since 1960)')
ax[1].set_ylabel('monthly performance')

sns.barplot(x=ind_grouped.index,y=ind_grouped.contraction,order=order_contraction,palette='summer',ax=ax[2])
ax[2].set_title('Average monthly performance  US industries during CONTRACTION regimes (since 1960)')
ax[2].set_ylabel('monthly performance')

sns.barplot(x=ind_grouped.index,y=ind_grouped.recovery,order=order_recovery,palette='summer',ax=ax[3])
ax[3].set_title('Average monthly performance  US industries during RECOVERY regimes (since 1960)')
ax[3].set_ylabel('monthly performance')

sns.despine()
plt.show()
fig.savefig('monhtly_performance.png')

In [29]:
spx.index=spx.index.tz_localize(None)
spx_daily_returns=spx.pct_change().dropna()
spx_monthly_returns=(spx_daily_returns+1).resample('M').apply(np.prod)-1
spx_36_rolling=spx_monthly_returns.rolling(window=36).aggregate(annualized_return,per=12).dropna()


In [30]:
#download 48 ff industries
datasets=web.famafrench.get_available_datasets()
dataset_name=[set for set in datasets if '48' in set and 'Industry' in set][0]

industry_dict=web.DataReader(dataset_name,'famafrench',start='1930-01-01')
industries_df=industry_dict[0]
industries_df.columns=industries_df.columns.str.strip()
industries_df.columns=industries_df.columns.str.lower()
industries_df.index=industries_df.index.astype('datetime64[ns]')
industries_df=industries_df.resample('M').last()
industries_df=industries_df/100
industries_df.dropna(axis=0,inplace=True)
industries_df.drop(['soda','hlth','fabpr','guns','gold'],axis=1,inplace=True)

In [31]:
#rolling correlations

industries_df_corr=industries_df.rolling(window=36).corr()
industries_df_corr.index.names=['date','industry']
industries_df_corr.dropna(inplace=True)

In [32]:
industries_df_corr

In [35]:
#x is the correlation matrix
fig,ax=plt.subplots()
rolling_avg_ind_corr=industries_df_corr.groupby(level='date').apply(lambda x:x.values.mean())


#plot
rolling_avg_ind_corr.plot(secondary_y=True,figsize=(12,5),title='Rolling 36 month annual S&P500 return & average correlation across 48 US industries (1930 - now)',color='red',linewidth=1,label='36m rolling correlation',legend=True)

spx_36_rolling.plot(color='midnightblue',linestyle='--',linewidth=1,label='36m rolling annual S&P500 return',legend=True)

plt.ylim(0.3,0.8)
plt.ylabel('Correlation coef')









sns.despine()

In [34]:
spx=yf.download('^GSPC',start='1930-01-01')['Adj Close']
